# Notebook to demonstrate multiscale cleaning on the canonical "M31" source.

We simulate some VLA data for M31 and then use msclean to restore it.

In [ ]:
%matplotlib inline

import sys, os 
print(os.path.abspath('.'))
sys.path.append(os.path.join('..','..')) 

import pylab
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy
import scipy
import scipy.special

from matplotlib import pylab
from matplotlib import pyplot as plt

import pyfits

from crocodile.synthesis import *
from crocodile.simulate import *
from util.visualize import *
from msclean import *

from crocodile.synthesis import *


First get the M31 model from the crocodile data directory

Now generate and plot the uvw coordinates

In [ ]:
vlas=numpy.genfromtxt("../../data/models/VLA_A_hor_xyz.txt", delimiter=",") 
vlas_xyz = xyz_at_latitude(vlas, numpy.pi*34.0784/180.0)/30.0 
uvw=xyz_to_baselines(vlas_xyz, 
numpy.arange(-numpy.pi/2.0, +numpy.pi/2.0,0.1), 
numpy.pi/8)
plt.clf()
plt.plot(uvw[:,0],uvw[:,1], '.')
plt.plot(-uvw[:,0],-uvw[:,1], '.', color='red')
plt.xlabel('U (m)')
plt.ylabel('V (m)')
plt.title('UV coverage')

In [ ]:
m31model='../../data/models/M31.MOD'
hdulist = pyfits.open(m31model)
m31image=numpy.array(hdulist[0].data)
lam=4.0*numpy.abs(uvw).max()
cell=1.0/lam
theta=256*cell
plt.clf()
plt.subplot(1,2,1)
plt.imshow(m31image, cmap='rainbow', origin='lower')

print("Field of view = %.2f degrees, maximum baseline %.3f (m)" % (theta*180.0/numpy.pi, lam))
pointimage=m31image.copy()
pointimage[225,75]+=2.0
pointimage[25,100]+=4.0
plt.subplot(1,2,2)
plt.imshow(m31image, cmap='rainbow', origin='lower')


uvwsort,vobs=dopredict(theta, lam, p=uvw, modelimage=m31image, predfn=wslicfwd)



Plot the visibility as a function of uv distance

In [ ]:
uvdist=numpy.sqrt(uvwsort[:,0]*uvwsort[:,0]+uvwsort[:,1]*uvwsort[:,1])
plt.clf()
plt.subplot(2,2,1)
plt.plot(uvwsort[:,0], numpy.real(vobs), '.')
plt.xlabel('U (wavelengths)')
plt.ylabel('Real part Visibility')

plt.subplot(2,2,2)
plt.plot(uvwsort[:,0], numpy.imag(vobs), '.')
plt.xlabel('U (wavelengths)')
plt.ylabel('Imag part Visibility')

plt.subplot(2,2,3)
plt.plot(uvdist, numpy.abs(vobs), '.')
plt.xlabel('UV distance (wavelengths)')
plt.ylabel('Amp Visibility')

plt.subplot(2,2,4)
plt.plot(uvdist, numpy.angle(vobs), '.')
plt.xlabel('UV distance (wavelengths)')
plt.ylabel('Phase Visibility')

plt.show()

Next we make the dirty image and psf using a simple gridder, just to check if it looks plausible

In [ ]:
d,p=doimg(2.0*theta, lam, uvwsort, vobs, wslicimg)
plt.clf()
plt.subplot(1,2,1)
plt.imshow(p, cmap='rainbow', origin='lower')
plt.title('Model')
c=plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(d, cmap='rainbow', origin='lower')
plt.title('Dirty')
c=plt.colorbar()



Now we can do the deconvolution

In [ ]:
scales=[1,10,30]
ps, vs, m31cc, m31res=majorcycle(2.0*theta, lam, uvwsort, vobs, 0.1, 3, 100, 500, scales)

Finally show the results

In [ ]:
plt.clf()
plt.subplot(1,2,1)
plt.imshow(numpy.sqrt(numpy.abs(m31cc)), cmap='rainbow', origin='lower')
c=plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(numpy.sqrt(numpy.abs(m31res)), cmap='rainbow', origin='lower')
c=plt.colorbar()


